In [28]:
from moku.instruments import FrequencyResponseAnalyzer
import control as ct
import altair as alt
import pandas as pd
import numpy as np

In [29]:
#configure the frequency sweep
fra = FrequencyResponseAnalyzer('192.168.50.57', force_connect=True, platform_id=4) #192.168.50.97 Moku Pro 1 | 192.168.50.57 Moku Pro 3
fra.set_sweep(start_frequency=15e6, stop_frequency=20e6, num_points=512,
                averaging_time=1e-3, averaging_cycles=10, settling_cycles=10,
                settling_time=1e-3)
fra.set_output(1, 0.05)
fra.set_output_termination(1, "50Ohm")
fra.set_frontend(1, "50Ohm", "DC", "400mVpp")

{'coupling': 'DC', 'impedance': '50Ohm', 'range': '400mVpp'}

In [30]:
delay = fra.start_sweep() 
print(delay)
data = fra.get_data(wait_complete = True)

{'estimated_sweep_time': 1.02402964667155}


In [31]:
fra.relinquish_ownership()

In [32]:
df = pd.DataFrame(data = data['ch1'])

df['label'] = 'measured'

df['magnitude'] = 10**(df['magnitude']/20) #convert to magnitude

In [33]:
magnitudechart = alt.Chart(df).mark_line(clip=True).encode(
    x=alt.X('frequency:Q').scale(type="log",domain = (1.5e7,2e7)),
    y=alt.Y('magnitude:Q').scale(type="log"),#,domain = (1,1.5)),
    color='label:N',
).properties(
    width=400,
    height=200
)

phasechart = alt.Chart(df).mark_line().encode(
    x=alt.X('frequency:Q').scale(type="log",domain = (1.5e7,2e7)),
    y=alt.Y('phase:Q'),
    color='label:N',
).properties(
    width=400,
    height=100
)

alt.vconcat(magnitudechart,phasechart)#.transform_filter(alt.FieldRangePredicate(field='freq', range=[8e6, 1e8]))

alt.VConcatChart(...)

In [34]:
freq_diff = df['frequency'].diff()*2*np.pi #in rad/s
df['unwrapped'] = np.unwrap(df['phase']+180.,period=360.)
phase_diff = df['unwrapped'].diff()*2*np.pi/360 #in radians
df['delay'] = phase_diff/freq_diff

In [35]:
delaychart = alt.Chart(df).mark_line().encode(
    x=alt.X('frequency:Q').scale(type="log",domain = (1.5e7,2e7)),
    y=alt.Y('delay:Q'),
    color='label:N',
).properties(
    width=400,
    height=200
)
delaychart

alt.Chart(...)

In [36]:
df['delay'].median()*1e9 #in nanoseconds

-350.6844647598392